In [37]:
import torch
import numpy
import matplotlib.pyplot as plt
import pandas


#import saveSimulations


inputAmplitude = 3
projectionAmplitude = 1.2

#Setup optimizer
batchSize = 10
MoAFactor = 0.1
spectralFactor = 1e-5
maxIter = 5000
noiseLevel = 10
stateLossFactor = 1e-4
L2 = 1e-6
lr = 2e-3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
import importlib.util
import sys
import os
lembas_path = '/nobackup/users/hmbaghda/Software/avlant_LEMBASGPU'

def import_from_path(name, path):
    spec = importlib.util.spec_from_file_location(name, path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    spec.loader.exec_module(module)
    return module

bionetwork = import_from_path('bionetwork', os.path.join(lembas_path, 'bionetwork.py'))
plotting = import_from_path('plotting', os.path.join(lembas_path, 'plotting.py'))

In [38]:
#Load network
networkList, nodeNames, modeOfAction = bionetwork.loadNetwork(os.path.join(lembas_path, 'data', 'macrophage-Model.tsv'))
annotation = pandas.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Annotation.tsv'), sep='\t')
uniprot2gene = dict(zip(annotation['code'], annotation['name']))
bionetParams = bionetwork.trainingParameters(targetSteps = 100, maxSteps = 150, expFactor= 50, tolerance = 1e-5, leak=0.01)

inName = annotation.loc[annotation['ligand'],'code'].values
outName = annotation.loc[annotation['TF'],'code'].values
inName = numpy.intersect1d(nodeNames, inName)
outName = numpy.intersect1d(nodeNames, outName)
outNameGene = [uniprot2gene[x] for x in outName]
nodeNameGene = [uniprot2gene[x] for x in nodeNames]

ligandInput = pandas.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Ligands.tsv'), sep='\t', low_memory=False, index_col=0)
TFOutput = pandas.read_csv(os.path.join(lembas_path, 'data', 'macrophage-TFs.tsv'), sep='\t', low_memory=False, index_col=0)

#Subset input and output to intersecting nodes
inName = ligandInput.columns.values
outName = TFOutput.columns.values

# doesnt change anything
inName = numpy.intersect1d(nodeNames, inName)
outName = numpy.intersect1d(nodeNames, outName)

# map from uniprot ID to gene name 
inNameGene = [uniprot2gene[x] for x in inName]
outNameGene = [uniprot2gene[x] for x in outName]

# c
ligandInput = ligandInput.loc[:,inName] # conditions x ligand (values are binary of presence or absence)
TFOutput = TFOutput.loc[:,outName] # conditions x TFs (values are TF activity score)
sampleName = ligandInput.index.values


In [39]:
model = bionetwork.model(networkList, nodeNames, modeOfAction, inputAmplitude, projectionAmplitude, inName, outName, bionetParams, 'MML')
model.inputLayer.weights.requires_grad = False
model.network.preScaleWeights()
model.setDevice(device)
model = model.to(device)

In [40]:
spectralCapacity = model.network.param['spectralTarget']

X = torch.tensor(ligandInput.values.copy(), dtype=torch.float32)
Y = torch.tensor(TFOutput.values, dtype=torch.float32)
X = X.to(device)
Y = Y.to(device)

In [51]:
os.path.abspath(os.curdir)

'/home/hmbaghda/Projects/scLEMBAS/scLEMBAS/model'